In [1]:
# set env variable to json credential file of google cloud

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'my_key_file.json'

In [2]:
from google.cloud import bigquery
import time
import pandas as pd
import numpy as np

In [3]:
# setup client
bq_client = bigquery.Client() 

In [34]:
# GETS SQL QUERY FOR 100 USERS (14,000-14,100) FROM SORTED TABLE OF MOST COMMENTS
# WHERE TOTAL COMMENTS IS SLIGHTLY ABOVE 100

QUERY = '''
        SELECT E.*, C.comments_count
        FROM `bigquery-public-data.hacker_news.full_201510` as E
        JOIN(
            SELECT *
            FROM(
                SELECT *
                FROM(
                    SELECT  `bigquery-public-data.hacker_news.full_201510`.by, COUNT(*) as comments_count
                    FROM `bigquery-public-data.hacker_news.full_201510`
                    GROUP BY `bigquery-public-data.hacker_news.full_201510`.by
                )  
                ORDER BY comments_count DESC
                LIMIT 14100
            )
            ORDER BY comments_count 
            LIMIT 100            
        ) C 
        ON E.by = C.by
        
        '''

In [35]:
import time
start = time.time()
df = bq_client.query(QUERY).to_dataframe()
end = time.time()
print(end - start)

8.250278234481812


In [36]:
df.shape

(12811, 14)

In [37]:
df.head()

,by,score,time,title,type,url,text,parent,deleted,dead,descendants,id,ranking,comments_count
0,kiallmacinnes,NaN,1398006060,None,comment,None,"I have to agree with the parent post, at least...",7616671.0,None,None,NaN,7616955,None,129
1,aaronchall,NaN,1423799051,None,comment,None,They&#x27;re in an incredibly good position. M...,9039278.0,None,None,NaN,9042768,None,128
2,fmoralesc,NaN,1367890389,None,comment,None,Whoa! These cadence utils are great! Bridging ...,5665492.0,None,None,NaN,5666009,None,128
3,jvzr,NaN,1367910960,None,comment,None,Could you try and sell it a bit more energetic...,5666831.0,None,None,NaN,5666869,None,128
4,namityadav,NaN,1307642938,None,comment,None,I know the founders of FeeFighters. I've sent ...,2637902.0,None,None,NaN,2638110,None,128


In [38]:
df['by'].value_counts().head(10)

WaxProlix        129
aiurtourist      129
adam-_-          129
jayair           129
kiallmacinnes    129
woobar           129
nancyhua         129
colinhowe        129
jfroma           129
bilban           129
Name: by, dtype: int64

In [79]:
# DATA WRANGLING

# Drop unecessary columns
df_drop = df.drop(columns= ['score','title','url','deleted','dead','descendants','ranking'])

# Fix weird text encodings
import html
df_drop['text']=df_drop['text'].apply(str)
df_drop['text'] = df_drop['text'].apply(lambda x: html.unescape(x))

# Remove html tags from string
import re
def remove_html_tags(text):
   clean = re.compile('<.*?>')
   return re.sub(clean, '', text)
df_drop['text'] = df_drop['text'].apply(lambda x: remove_html_tags(x))


# Convert unix time to datetime object with accuracy of secs
from datetime import datetime
df_drop['time']=pd.to_datetime(df_drop['time'],unit='s')

In [80]:
df_drop.head()

,by,time,type,text,parent,id,comments_count
0,kiallmacinnes,2014-04-20 15:01:00,comment,"I have to agree with the parent post, at least...",7616671.0,7616955,129
1,aaronchall,2015-02-13 03:44:11,comment,They're in an incredibly good position. My for...,9039278.0,9042768,128
2,fmoralesc,2013-05-07 01:33:09,comment,Whoa! These cadence utils are great! Bridging ...,5665492.0,5666009,128
3,jvzr,2013-05-07 07:16:00,comment,Could you try and sell it a bit more energetic...,5666831.0,5666869,128
4,namityadav,2011-06-09 18:08:58,comment,I know the founders of FeeFighters. I've sent ...,2637902.0,2638110,128


In [81]:
# Populate sentiment analysis columns in dataframe
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [82]:
df_drop['neg']=np.zeros(df_drop.shape[0])
df_drop['pos']=np.zeros(df_drop.shape[0])
df_drop['neu']=np.zeros(df_drop.shape[0])

In [83]:
# Adding sentiments to dataframe is computationally intensive
# On Martin's local computer 12,811 comments took ~24 min 

import time
start = time.time()

# populate vader sentiments in additional cols
for idx,x in df_drop['text'].iteritems():
    #print (idx,x)
    
    df_drop['neg'][idx] = sia.polarity_scores(x)['neg']
    df_drop['pos'][idx] = sia.polarity_scores(x)['pos']
    df_drop['neu'][idx] = sia.polarity_scores(x)['neu']    
end = time.time()
print(end - start)

/home/martin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/martin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/martin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


KeyboardInterrupt: 

In [84]:
df_drop.head()

,by,time,type,text,parent,id,comments_count,neg,pos,neu
0,kiallmacinnes,2014-04-20 15:01:00,comment,"I have to agree with the parent post, at least...",7616671.0,7616955,129,0.040,0.131,0.829
1,aaronchall,2015-02-13 03:44:11,comment,They're in an incredibly good position. My for...,9039278.0,9042768,128,0.102,0.117,0.781
2,fmoralesc,2013-05-07 01:33:09,comment,Whoa! These cadence utils are great! Bridging ...,5665492.0,5666009,128,0.000,0.198,0.802
3,jvzr,2013-05-07 07:16:00,comment,Could you try and sell it a bit more energetic...,5666831.0,5666869,128,0.000,0.245,0.755
4,namityadav,2011-06-09 18:08:58,comment,I know the founders of FeeFighters. I've sent ...,2637902.0,2638110,128,0.000,0.257,0.743


In [85]:
# Shrink the dataframe to 100 comments per user, where comment is not a pure neg or pos so that
# comment displayed is somewhat interesting and not just one word.

In [86]:
users = df_drop['by'].unique()
len(users)

100

In [87]:
df_final = pd.DataFrame(data=None, columns=df_drop.columns)
for user in users:
    # Create a dataframe of only one user
    df_user = df_drop[df_drop['by'] == user]
    
    # Remove pure sentiments
    df_user_unpure = df_user[(df_user['neg'] !=1) & (df_user['pos'] !=1)]
    
    # Submit first 100 comments of the unpure sentiments to final dataframe
    df_final = df_final.append(df_user_unpure[:100])
    
df_final.shape

(10000, 10)

In [93]:
df_final.head(10)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
0,kiallmacinnes,2014-04-20 15:01:00,comment,"I have to agree with the parent post, at least...",7616671.0,7616955,129,0.040,0.131,0.829
55,kiallmacinnes,2015-06-19 22:24:54,comment,"Agreed, most people hear ""rewrite history"" and...",9745350.0,9748039,129,0.020,0.123,0.858
60,kiallmacinnes,2013-11-11 11:45:28,comment,This is awful - Almost as bad as breaking the ...,6710992.0,6711110,129,0.409,0.000,0.591
71,kiallmacinnes,2013-06-17 09:27:34,comment,"No, using a firewall increases security. It ju...",5892138.0,5892195,129,0.089,0.108,0.803
108,kiallmacinnes,2012-10-09 13:36:44,comment,The fact that the fake nameservers were visibl...,4631372.0,4631392,129,0.094,0.000,0.906
166,kiallmacinnes,2012-10-29 17:22:04,comment,Isn't that exactly what they did?e.g. X.Y.X wh...,4713127.0,4713318,129,0.000,0.000,1.000
275,kiallmacinnes,2013-05-24 10:58:12,comment,"I would pick a project you use, a framework/li...",5762061.0,5762094,129,0.000,0.000,0.000
560,kiallmacinnes,2013-10-31 19:42:40,comment,Well.. that was probably the most impulsive bu...,6648493.0,6649207,129,0.000,0.000,0.000
568,kiallmacinnes,2012-09-14 17:06:01,comment,"[2]'s comment ""you get access to the same web ...",4522241.0,4522682,129,0.000,0.000,0.000
598,kiallmacinnes,2013-12-09 11:48:39,comment,"Have to admit, I'm a little surprised at just ...",6873032.0,6873875,129,0.000,0.000,0.000


In [89]:
# Random comments from the 100 users
df_random100 = df_final.sample(100)
df_random100.head(10)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
463,gruez,2015-06-20 18:25:58,comment,"I'm divided on this. On one hand, if everyone ...",9750572.0,9750636,128,0.0,0.0,0.0
8004,MatthewWilkes,2015-06-11 10:15:05,comment,Have you ever tried using one? Most places are...,9698706.0,9698724,128,0.0,0.0,0.0
8169,VieElm,2014-11-19 06:25:36,story,,NaN,8628559,128,0.0,0.0,0.0
3299,mmackh,2011-10-28 10:21:09,comment,"Thanks! When you tap Clear Read, the article's...",3167613.0,3167682,128,0.0,0.0,0.0
2246,ukigumo,2015-02-08 15:14:05,comment,I wrote about the different worlds of people t...,9017148.0,9017417,128,0.0,0.0,0.0
494,duncans,2014-09-19 06:36:42,comment,Nitpick: they should have been encoding the ou...,8338718.0,8339229,128,0.0,0.0,0.0
2968,maxsilver,2013-09-20 20:53:56,comment,I think the parent's mistaken BB7 with BB10. W...,6420388.0,6420446,128,0.0,0.0,0.0
2724,imaginator,2014-09-04 22:04:59,comment,Agree that it's shortsightedness.But I think t...,8270469.0,8270938,128,0.0,0.0,0.0
4827,joncooper,2014-01-10 22:08:39,comment,"As you say, the strong social justice momentum...",7029353.0,7039877,128,0.0,0.0,0.0
5873,vm,2011-12-14 06:03:09,story,,NaN,3350843,128,0.0,0.0,0.0


In [90]:
# Most negative comments
df_neg100 = df_final.sort_values(['neg'],ascending=False)[:100]
df_neg100.head(10)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
60,kiallmacinnes,2013-11-11 11:45:28,comment,This is awful - Almost as bad as breaking the ...,6710992.0,6711110,129,0.409,0.000,0.591
23,lfuller,2013-10-05 03:45:28,comment,I'm glad someone else mentioned this. I have d...,6496663.0,6499309,128,0.309,0.110,0.581
151,codingthewheel,2010-04-30 11:30:27,comment,As much as I dislike Apple's current philosoph...,1305346.0,1307833,128,0.300,0.000,0.700
66,jaboutboul,2014-01-09 17:35:55,comment,This is pretty damn cool,7031377.0,7031832,128,0.265,0.539,0.196
125,burnte,2015-09-27 03:42:34,comment,"Oh, I'm all for the ""let it crash"" ideology, j...",10281111.0,10285369,128,0.254,0.090,0.657
118,taprun,2014-07-15 19:30:14,comment,Learn how to analyze a business problem and co...,8034957.0,8038475,128,0.238,0.107,0.654
79,colinhowe,2011-04-03 07:21:17,comment,"The ""best"" is way overengineered. I doubt this...",2401929.0,2402056,129,0.238,0.000,0.762
19,imaginator,2010-10-21 07:34:17,comment,You are right. I was confusing Symbian^1 with ...,1812704.0,1814603,128,0.220,0.074,0.706
165,judegomila,2009-05-31 09:20:50,comment,Good idea. I'm going to superset fear with cor...,634569.0,634571,128,0.203,0.354,0.443
58,VieElm,2014-11-20 16:13:15,comment,"I don't want joy, I want to be able to see my ...",8636061.0,8636402,128,0.197,0.090,0.713


In [91]:
# Most positive comments
df_pos100 = df_final.sort_values(['pos'],ascending=False)[:100]
df_pos100.head(10)

,by,time,type,text,parent,id,comments_count,neg,pos,neu
112,bilban,2011-03-09 16:39:06,comment,"Oh well, I at least appreciated the humour.",2305207.0,2305368,129,0.000,0.680,0.320
66,jaboutboul,2014-01-09 17:35:55,comment,This is pretty damn cool,7031377.0,7031832,128,0.265,0.539,0.196
34,gregable,2010-12-07 09:02:16,comment,"Yes, my comment is a clear rip off of MMM.",1978243.0,1978488,128,0.000,0.431,0.569
162,differentView,2015-03-17 19:49:30,comment,What's your definition of popular?,9220665.0,9220816,128,0.000,0.412,0.588
165,judegomila,2009-05-31 09:20:50,comment,Good idea. I'm going to superset fear with cor...,634569.0,634571,128,0.203,0.354,0.443
50,gametheoretic,2014-01-07 03:33:02,comment,"Avoid anything that starts with 'Why'. ""Gather...",7015128.0,7015247,128,0.111,0.337,0.553
169,cpswan,2012-01-26 14:21:43,comment,It would be nice if the inspired by line actua...,3513559.0,3514205,128,0.000,0.333,0.667
41,pama,2011-02-10 02:25:20,comment,Thanks for posting -- this is an amazing resou...,2200309.0,2200411,128,0.000,0.312,0.688
160,pama,2010-10-20 13:21:37,comment,It also helps recall content better than when ...,1810857.0,1811059,128,0.000,0.294,0.706
11,fmoralesc,2011-11-02 16:56:32,comment,Maybe creating educational institutions that t...,3187633.0,3187721,128,0.000,0.278,0.722


In [92]:
# Save to local json

#df_final.to_json(r'tiny.json')
#df_random100.to_json(r'tiny_random100.json')
#df_neg100.to_json(r'tiny_neg100.json')
#df_pos100.to_json(r'tiny_pos100.json')               